# Run high throughput screening calculation: single point calculation

## Goal of this tutorial

This tutorial demonstrates **batch processing** of multiple structures:
- Run single point calculations on all structure files in a folder
- Organize all calculations in an AiiDA group for easy management
- Check calculation status and results in bulk
- Visualize the provenance

## Prerequisites

Before running this notebook, you need:
1. A folder with structure files (`.cif` format)
2. The `config_sp.yaml` file in the same directory as this notebook
3. A configured AiiDA code (`janus@localhost`)

While creating a group is not mandatory, it makes handling large amounts of data much easier.

In [ ]:
! verdi  group create "hts_calc"

In [ ]:
! verdi group show "hts_calc"

Import all the needed modules

In [ ]:
import click
from aiida.engine import run_get_node, submit, run, run_get_pk
from aiida.orm import load_code, load_node, load_group
from aiida.plugins import CalculationFactory
from pathlib import Path
from aiida_mlip.data.model import ModelData
from aiida_mlip.data.config import JanusConfigfile
from aiida_mlip.helpers.help_load import load_structure
import csv
import sys
from aiida.common import NotExistent
import time
from aiida import load_profile
load_profile()

Define inputs for the calculation. All the inputs are in common for all the structures, except the StructureData inputs which is gonna be defined in a loop cycle through the folder.

In [ ]:
Calculation = CalculationFactory("mlip.sp")
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp")
# model = ModelData.from_local("path/to/model", architecture="mace_mp")
metadata = {"options": {"resources": {"num_machines": 1}}}
code = load_code("janus@localhost")

Here we use a config file to pass the other parameters. This file should be in the same directory as the notebook.

In [ ]:
conf = JanusConfigfile(Path("config_sp.yaml").absolute())

Load the group as a variable:

In [ ]:
group = load_group("hts_calc")

Define the folder containing the structure files. 

Modify this path to point to your structure files. For example, to use the example structures from the repository:

In [ ]:
p = Path("structures")

Now we define for each file in the folder the `StructureData` and run the calculation. In this case we use the command `run_get_pk` to run, but when submitting large amounts of calculations, especially if they are more time consuming than the single point calculation, it is better to submit to the queue with the command `submit`.

In [ ]:
list_of_nodes = []
calculation_count = 0

for child in p.glob('**/*'):
    if child.name.endswith("cif"):   
        print(f"\nProcessing: {child.name}")
        metadata['label']=f"{child.name}"
        # This structure will overwrite the one in the config file if present
        structure = load_structure(child.absolute())
        # Run calculation
        result, pk = run_get_pk(
            Calculation,
            code=code,
            struct=structure,
            metadata=metadata,
            config=conf,
            model=model
        )
        list_of_nodes.append(pk)
        node = load_node(pk)
        group.add_nodes(node)
        calculation_count += 1
        time.sleep(1)
        print(f"Calculation finished with PK: {pk}")

print(f"\n{'='*60}")
print(f"FINISHED: Processed {calculation_count} structures")
print(f"All calculation PKs: {list_of_nodes}")
print(f"{'='*60}")

If we look at the group now, it will contain the calculations:

In [ ]:
! verdi group show "hts_calc"

We can check if the calculations finished correctly. Exit status should be 0.

In [ ]:
for node in group.nodes:
    print(f"Calculation {node.label, node.pk} finished with exit status {node.exit_status}" )

Here, if we generate the provenance graph of a calculation we will see that it shows that calculation only, not the others. That is because the calculations are disconnected from each other, even if we submitted them together using the same inputs.

In [ ]:
! verdi node graph generate {node.pk}

In the `geomopt.py` tutorial I show a way to "connect" calculations to make them appear in the provenance graph together. Otherwise, the high-throughput screening can be set up as a AiiDA `WorkChain` for this purpose. This WorkChain is currently a work in progress and will be added to the plugin.

We can interact with the nodes by playing around with the functions of the node.
Here I am deleting the node to empty the group (which is useful for the tutorial but not something you want to do normally, especially if the data is good)

In [ ]:
from aiida.tools import delete_nodes

for node in group.nodes:
    pks_to_be_deleted = delete_nodes(
        [node.pk], dry_run=False, create_forward=True, call_calc_forward=True, call_work_forward=True
    )

In [ ]:
! verdi group show "hts_calc"


In [ ]:
! verdi process list -a